<img src="../images/esgf.png" width=250 alt="CMIP6 image"></img>
<img src="../images/cmip6-cookbook-thumbnail.png" width=250 alt="CMIP6 image"></img>

# Search and Load CMIP6 Data via ESGF/OPeNDAP

---

## Overview

This notebook shows how to search and load data via [Earth System Grid Federation](https://esgf.llnl.gov) infrastructure. This infrastructure works great and is the foundation of the CMIP6 distribution system.

The main technologies used here are the [ESGF search API](https://github.com/ESGF/esgf.github.io/wiki/ESGF_Search_REST_API), used to figure out what data we want, and [OPeNDAP](https://www.opendap.org/), a remote data access protocol over HTTP.

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Intro to Xarray](https://foundations.projectpythia.org/core/xarray/xarray-intro.html) | Necessary | |
| [Understanding of NetCDF](https://foundations.projectpythia.org/core/data-formats/netcdf-cf.html) | Helpful | Familiarity with metadata structure |

- **Time to learn**: 10 minutes

---

## Imports

In [ ]:
import warnings

from distributed import Client
import holoviews as hv
import hvplot.xarray
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from pyesgf.search import SearchConnection
import xarray as xr

xr.set_options(display_style='html')
warnings.filterwarnings("ignore")
hv.extension('bokeh')

In [ ]:
client = Client()
client

## Search using ESGF API

Fortunately, there is an ESGF API implemented in Python - `pyesgf`, which requires three major steps:
- Establish a search connection
- Query your data
- Extract the urls to your data

Once you have this information, you can load the data into an `xarray.Dataset`.

### Configure the connection to a data server
First, we configure our connection to some server, using the distributed option (`distrib=False`). In this case, we are searching from the Lawerence Livermore National Lab (LLNL) data node.

In [ ]:
conn = SearchConnection('https://esgf-node.llnl.gov/esg-search',
                        distrib=False)

### Query our dataset
We are interested in a single experiment from CMIP6 - one of the Community Earth System Model version 2 (CESM2) runs, specifically the historical part of the simulation.

We are also interested in a single variable - temperature at the surface (**tas**), with a single ensemble member (`r10i1p1f1`)

In [ ]:
ctx = conn.new_context(
    facets='project,experiment_id',
    project='CMIP6',
    table_id='Amon',
    institution_id="NCAR",
    experiment_id='historical',
    source_id='CESM2',
    variable='tas',
    variant_label='r10i1p1f1',
)

### Extract the OpenDAP urls
In order to access the datasets, we need the urls to the data. Once we have these, we can read the data remotely!

In [ ]:
result = ctx.search()[0]
files = result.file_context().search()
files

The files object is not immediately helpful - we need to extract the `opendap_url` method from this.

In [ ]:
files[0].opendap_url

We can use this for the whole list using list comprehension, as shown below.

In [ ]:
opendap_urls = [file.opendap_url for file in files]
opendap_urls

## Read the data into an `xarray.Dataset`
Now that we have our urls to the data, we can use open multifile dataset (`open_mfdataset`) to read the data, combining the coordinates and chunking by time.

Xarray, together with the netCDF4 Python library, allow lazy loading.

In [ ]:
ds = xr.open_mfdataset(opendap_urls,
                       combine='by_coords',
                       chunks={'time':480})
ds

### Plot a quick look of the data
Now that we have the dataset, let's plot a few quick looks of the data.

In [ ]:
ds.tas.sel(time='1950-01').squeeze().plot(cmap='Spectral_r');

These are OPeNDAP endpoints. Xarray, together with the netCDF4 Python library, allow lazy loading.

## Compute an area-weighted global average
Let's apply some computation to this dataset. We would like to calculate the global average temperature. This requires weighting each of the grid cells properly, using the area.

### Find the area of the cells

We can query the dataserver again, this time extracting the area of the cell (`areacella`).

In [ ]:
ctx = conn.new_context(
    facets='project,experiment_id',
    project='CMIP6',
    institution_id="NCAR",
    experiment_id='historical',
    source_id='CESM2',
    variable='areacella',
)

As before, we extract the opendap urls.

In [ ]:
result = ctx.search()[0]
files = result.file_context().search()
opendap_urls = [file.opendap_url for file in files]
opendap_urls

And finally, we load our cell area file into an `xarray.Dataset`

In [ ]:
ds_area = xr.open_dataset(opendap_urls[0])
ds_area

### Compute the global average
Now that we have the area of each cell, and the temperature at each point, we can compute the global average temperature.

In [ ]:
total_area = ds_area.areacella.sum(dim=['lon', 'lat'])
ta_timeseries = (ds.tas * ds_area.areacella).sum(dim=['lon', 'lat']) / total_area
ta_timeseries

By default the data are loaded lazily, as Dask arrays. Here we trigger computation explicitly.

In [ ]:
%time ta_timeseries.load()

### Visualize our results
Now that we have our results, we can visualize using static and dynamic plots. Let's start with static plots using `matplotlib`, then dynamic plots using `hvPlot`.

In [ ]:
ta_timeseries['time'] = ta_timeseries.indexes['time'].to_datetimeindex()

In [ ]:
fig = plt.figure(figsize=(12,8))
ta_timeseries.plot(label='monthly')
ta_timeseries.rolling(time=12).mean().plot(label='12 month rolling mean')
plt.legend()
plt.title('Global Mean Surface Air Temperature')

In [ ]:
ta_timeseries.name = 'Temperature (K)'
monthly_average = ta_timeseries.hvplot(title = 'Global Mean Surface Air Temperature',
                                       label='monthly')
rolling_monthly_average = ta_timeseries.rolling(time=12).mean().hvplot(label='12 month rolling mean',)

(monthly_average * rolling_monthly_average).opts(legend_position='top_left')

---

## Summary
In this notebook, we searched for and opened a CESM2 dataset using the ESGF API and OPeNDAP. We then plotted global average surface air temperature.

### What's next?
We will see some more advanced examples of using the CMIP6 data.

## Resources and references
- [Original notebook in the Pangeo Gallery](http://gallery.pangeo.io/repos/pangeo-gallery/cmip6/search_and_load_with_esgf_opendap.html) by Henri Drake and [Ryan Abernathey](https://ocean-transport.github.io/)